## Practica 2

### Ejercicio 1)
¿En el dataset del ejercicio 1 de la práctica 1 indique para cada Job, si se vería beneficiado por una función combiner?
En caso afirmativo, ¿cuál es la implementación de dicha función? ¿Qué datos recibe cada reduce, al utilizar la función combiner?

### 2) Implemente una función combiner para el problema del WordCount.

In [1]:
from MRE import Job
root_path = './'
inputDir = root_path + "WordCount/input/"
outputDir = root_path + "WordCount/ejercicio_2/"

def fmap(key, value, context):
    words = value.split()
    
    for w in words:
        context.write(w, 1)
        
def fred(key, values, context):
    c=0
    for v in values:
        c=c+1
    context.write(key, c)

job = Job(inputDir, outputDir, fmap, fred)
job.setCombiner(fred)
success = job.waitForCompletion()
print(success)

[MRE] Iniciando etapa de mapeo...
[Map] EL -> 1
[Map] AMO -> 1
[Map] VIEJO -> 1
[Map] Nos -> 1
[Map] apeamos -> 1
[Map] de -> 1
[Map] nuestras -> 1
[Map] caballerías -> 1
[Map] en -> 1
[Map] un -> 1
... [más resultados de map omitidos]
[MRE] Iniciando etapa de reducción...
[Reduce] Clave recibida: !ah!
[Reduce] Clave recibida: "A"
[Reduce] Clave recibida: "B"
[Reduce] Clave recibida: "Dr.
[Reduce] Clave recibida: "El
[Reduce] Clave recibida: "purgar",
[Reduce] Clave recibida: (aquÃ­
[Reduce] Clave recibida: (con
[Reduce] Clave recibida: (naturalmente)
[Reduce] Clave recibida: *
... [más reduce claves omitidas]
[MRE] Finalizando y escribiendo resultados en disco...

[MRE] Resultados finales del Job:
!ah!	1
"A"	1
"B"	1
"Dr.	1
"El	1
"purgar",	1
(aquÃ­	1
(con	1
(naturalmente)	1
*	5
... [más resultados omitidos]
True


### Ejercicio 3)
Implemente un job MapReduce para calcular el máximo, mínimo, promedio y desvío stándard de las ocurrencias de todas las palabras del dataset Libros.